# Calendar

Upcoming Research Coding Community workshops and events are organised below.


Events are read from `data/events.csv`. Ensure the file has the columns `title`, `club`, `start`, `end`, `location`, and `description`, with date-times in ISO format such as `2024-10-07 13:00`. Only events scheduled from today onward are shown.


In [ ]:
import csv
import html
from collections import defaultdict
from datetime import datetime, timedelta
from pathlib import Path
from uuid import uuid4

from IPython.display import HTML, Markdown, display

DATA_PATH = Path('data') / 'events.csv'
REQUIRED_COLUMNS = ['title', 'club', 'start', 'end', 'location', 'description']
PALETTE = ['#1976d2', '#2e7d32', '#8e24aa', '#ef6c00', '#00838f', '#5d4037']

def load_events(path: Path):
    if not path.exists():
        display(Markdown(f"Warning: `{path}` is missing. Add it to populate the calendar."))
        return []

    with path.open('r', encoding='utf-8-sig') as handle:
        reader = csv.DictReader(handle)
        if not reader.fieldnames:
            display(Markdown('Warning: events file has no header row.'))
            return []

        missing = [col for col in REQUIRED_COLUMNS if col not in reader.fieldnames]
        if missing:
            display(Markdown('Warning: events file is missing columns: ' + ', '.join(missing)))
            return []

        events = []
        for row in reader:
            try:
                start = datetime.fromisoformat(row['start'].strip())
                end = datetime.fromisoformat(row['end'].strip())
            except (ValueError, AttributeError):
                continue
            if end <= start:
                continue
            events.append({
                'title': row['title'].strip() or 'Untitled event',
                'club': row['club'].strip() or 'Community',
                'start': start,
                'end': end,
                'location': row['location'].strip() or 'Location to be confirmed',
                'description': row['description'].strip(),
            })

    today = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
    future_events = [evt for evt in events if evt['end'] >= today]
    future_events.sort(key=lambda item: item['start'])
    if not future_events:
        display(Markdown('No upcoming events were found.'))
    return future_events

def allocate_colors(events):
    club_map = {}
    for evt in events:
        if evt['club'] not in club_map:
            club_map[evt['club']] = PALETTE[len(club_map) % len(PALETTE)]
    return club_map

def month_start(date_obj):
    return date_obj.replace(day=1, hour=0, minute=0, second=0, microsecond=0)

def build_months(events):
    months = defaultdict(list)
    for evt in events:
        key = month_start(evt['start'])
        months[key].append(evt)
    return dict(sorted(months.items()))

def render_month(month_key, month_events, colors):
    month_label = month_key.strftime('%B %Y')
    start_of_grid = month_key - timedelta(days=month_key.weekday())
    days = [start_of_grid + timedelta(days=offset) for offset in range(42)]

    html_days = []
    today_date = datetime.today().date()
    events_by_day = defaultdict(list)
    for evt in month_events:
        events_by_day[evt['start'].date()].append(evt)

    for day in days:
        classes = ['calendar-day']
        if day.month != month_key.month:
            classes.append('outside')
        if day.date() == today_date:
            classes.append('today')
        day_events = events_by_day.get(day.date(), [])

        event_html = []
        for evt in day_events:
            color = colors.get(evt['club'], '#1976d2')
            start_time = evt['start'].strftime('%H:%M')
            end_time = evt['end'].strftime('%H:%M')
            bits = [f"<span class="time">{html.escape(start_time)}-{html.escape(end_time)}</span>"]
            bits.append(f"<span class="title">{html.escape(evt['title'])}</span>")
            if evt['location']:
                bits.append(f"<span class="location">{html.escape(evt['location'])}</span>")
            event_html.append(
                "<div class="calendar-event" style="background-color: {color};">" + ' '.join(bits) + "</div>"
            )

        cell = [
            f"<div class="date-label">{day.day}</div>",
            *event_html
        ]
        html_days.append(f"<div class="{' '.join(classes)}">{''.join(cell)}</div>")

    return month_label, ''.join(html_days)

def build_calendar(events):
    if not events:
        return

    months = build_months(events)
    colors = allocate_colors(events)
    container_id = f"calendar-{uuid4().hex}"

    month_blocks = []
    month_options = []

    for month_key, month_events in months.items():
        label, grid_html = render_month(month_key, month_events, colors)
        value = month_key.strftime('%Y-%m')
        month_options.append(f"<option value="{value}">{html.escape(label)}</option>")
        month_blocks.append(
            f"<div class="calendar-month" data-month="{value}">"
            f"<div class="calendar-grid">{grid_html}</div>"
            f"</div>"
        )

    legend_items = []
    for club, color in colors.items():
        legend_items.append(
            f"<span><span class="color-swatch" style="background-color: {color};"></span>{html.escape(club)}</span>"
        )

    style_block = """
<style>
.calendar-container { font-family: 'Inter', 'Helvetica', 'Arial', sans-serif; margin: 1rem 0; }
.calendar-controls { display: flex; align-items: center; gap: 0.5rem; margin-bottom: 0.75rem; }
.calendar-controls label { font-weight: 600; }
.calendar-month { display: none; }
.calendar-month.active { display: block; }
.calendar-grid { display: grid; grid-template-columns: repeat(7, minmax(130px, 1fr)); border: 1px solid #d0d7de; border-bottom: none; border-right: none; }
.calendar-day { border-right: 1px solid #d0d7de; border-bottom: 1px solid #d0d7de; min-height: 120px; padding: 6px; box-sizing: border-box; font-size: 0.8rem; background-color: #fff; }
.calendar-day:last-child { border-right: none; }
.calendar-day.outside { background-color: #f6f8fa; color: #657786; }
.calendar-day.today { box-shadow: inset 0 0 0 2px #1976d2; }
.date-label { font-weight: 600; margin-bottom: 0.25rem; }
.calendar-event { color: #fff; border-radius: 4px; padding: 4px 6px; margin-bottom: 4px; line-height: 1.2; }
.calendar-event .time { display: block; font-weight: 600; }
.calendar-event .title { display: block; }
.calendar-event .location { display: block; font-size: 0.72rem; opacity: 0.9; }
.calendar-legend { display: flex; flex-wrap: wrap; gap: 0.75rem; margin-top: 0.75rem; font-size: 0.85rem; align-items: center; }
.calendar-legend .color-swatch { width: 12px; height: 12px; border-radius: 50%; display: inline-block; }
@media (max-width: 900px) { .calendar-grid { grid-template-columns: repeat(2, minmax(150px, 1fr)); } }
</style>
"""

    script_block = f"""
<script>
(function() {{
    var container = document.getElementById('{container_id}');
    if (!container) return;
    var select = container.querySelector('select');
    var months = container.querySelectorAll('.calendar-month');
    function showMonth(value) {{
        months.forEach(function(block) {{
            if (block.dataset.month === value) {{
                block.classList.add('active');
            }} else {{
                block.classList.remove('active');
            }}
        }});
    }}
    select.addEventListener('change', function(evt) {{ showMonth(evt.target.value); }});
    if (select.value) {{
        showMonth(select.value);
    }} else if (months.length) {{
        select.value = months[0].dataset.month;
        showMonth(select.value);
    }}
}})();
</script>
"""

    html_block = (
        style_block +
        f"<div id="{container_id}" class="calendar-container">" +
        f"<div class="calendar-controls"><label for="{container_id}-select">Month:</label>" +
        f"<select id="{container_id}-select" aria-label="Select month">{''.join(month_options)}</select>" +
        "</div>" +
        ''.join(month_blocks) +
        (f"<div class="calendar-legend">{''.join(legend_items)}</div>" if legend_items else '') +
        "</div>" +
        script_block
    )

    display(HTML(html_block))

events = load_events(DATA_PATH)
build_calendar(events)
